# Importsand preparations

In [1]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset, Dataset, DatasetDict

/itf-fi-ml/home/krimhau/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !watch -n 0.5 nvidia-smi

In [3]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

PyTorch version: 2.1.0+cu121
CUDA version: 12.1
cuDNN version: 8902
Current device: 0
Is cuda available: True


In [4]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

Transformers version: 4.35.0
Datasets version: 2.14.6


In [5]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [6]:
# Find the GPU with the least memory usage.
!nvidia-smi

Tue Nov  7 10:07:15 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-PCIE-40GB           On | 00000000:01:00.0 Off |                    0 |
| N/A   57C    P0              247W / 250W|  23612MiB / 40960MiB |     94%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [7]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 89% | 58% |
|  1 |  0% | 98% |
|  2 |  0% |  0% |
|  3 |  0% | 70% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 | 87% | 58% |
|  1 | 24% | 98% |
|  2 |  8% |  1% |
|  3 |  0% | 70% |


In [8]:
data = pd.read_csv("csv/priority_high_or_not_high_clean.csv" , index_col = 0)
data

,text_clean,label
0,implement an android image picker to browse im...,0
1,when a video is not playing hover should displ...,0
2,httpstwittercomdisneythingzstatus1224143546594...,0
3,description there has been a request from a nu...,0
4,description escape shuttle accelerates faster ...,0
...,...,...
539391,id 0x7b7d2f7b7d2f7b6578706f727465645f6e616d657...,1
539392,there is no link to the game from the childs p...,1
539393,i set up an alb with a cognito auth listener a...,1
539394,in bfe00c5httpsgithubcomxnaasnitterinstancesco...,1


In [9]:
# Smaller and faster than bert.
base_model_id = "distilbert-base-uncased"

epochs = 5 #Number of full cyles through the training set.
num_labels = 2 #Number of labels, high, med, low priority.
learning_rate = 5e-5 # Rate the model updates based on the data its trained on.
train_batch_size = 16 # Number of training examples in one iteration.
eval_batch_size = 32 # Number evalutaion examples in on iteratoion.
save_strategy = "no" # Should the model be saved automatically during training.
save_steps = 500 # How often to save the model during training. No effect since no over.
logging_steps = 100
model_dir = "./model" #Where to save model

# Use early stopping to prevent overfitting
#load_best_model_at_end=True
#metric_for_best_model="eval_loss"
#greater_is_better=False

In [10]:

# Split dataframe into three parts: training, validation and testing.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [11]:
# Drops rows with missing values
data.dropna(inplace=True)

In [12]:
# Resets the index after dropping rows
data.reset_index(inplace=True)
data

,index,text_clean,label
0,0,implement an android image picker to browse im...,0
1,1,when a video is not playing hover should displ...,0
2,2,httpstwittercomdisneythingzstatus1224143546594...,0
3,3,description there has been a request from a nu...,0
4,4,description escape shuttle accelerates faster ...,0
...,...,...,...
539391,539391,id 0x7b7d2f7b7d2f7b6578706f727465645f6e616d657...,1
539392,539392,there is no link to the game from the childs p...,1
539393,539393,i set up an alb with a cognito auth listener a...,1
539394,539394,in bfe00c5httpsgithubcomxnaasnitterinstancesco...,1


In [13]:
#Drops the index col, better for managint the data.
data.drop(columns= ["index"], inplace = True)

In [14]:
data

,text_clean,label
0,implement an android image picker to browse im...,0
1,when a video is not playing hover should displ...,0
2,httpstwittercomdisneythingzstatus1224143546594...,0
3,description there has been a request from a nu...,0
4,description escape shuttle accelerates faster ...,0
...,...,...
539391,id 0x7b7d2f7b7d2f7b6578706f727465645f6e616d657...,1
539392,there is no link to the game from the childs p...,1
539393,i set up an alb with a cognito auth listener a...,1
539394,in bfe00c5httpsgithubcomxnaasnitterinstancesco...,1


In [15]:
# 60% trainig, 20% validate, 20% test. Seed None.
# Test 80-10-10 and 70-15-15
train , validate , test = train_validate_test_split(data)

In [16]:
train.set_index("label" , inplace = True)
validate.set_index("label" , inplace = True)
test.set_index("label" , inplace = True)

In [17]:
test

,text_clean
label,
0,describe the bug the whole user guide is not c...
0,original issue 1101httpscodegooglecompcodename...
1,hi how do i set username and password textfiel...
1,on the home screen if a user doesnt have permi...
1,its super unlikely to be a problem but the azu...
...,...
0,browser firefox mobile 680 uaheader mozilla50 ...
1,a player object has the following properties n...
1,minecraft version 1171 forge version 3702 logs...


In [18]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
testds = Dataset.from_pandas(test)

ds = DatasetDict()

ds["test"] = testds
ds["train"] = tds
ds["validate"] = vds

ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 53941
    })
    train: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 431516
    })
    validate: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 53939
    })
})

In [19]:
train_dataset = ds["train"]
valid_dataset = ds["validate"]
test_ds = ds["test"]

In [20]:
ds["train"][0]

{'text_clean': 'in 19f8ba0httpsgithubcomkaratekhdstatuscommit19f8ba0124a1fc68c14a76c7f775f8eb10e68e72 iserv for comparison secretsite2 was down http code 0 response time 0 ms',
 'label': 1}

In [21]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(base_model_id, num_labels=num_labels)
#tokenizer = AutoTokenizer.from_pretrained(base_model_id)
# optim = torch.optim.Adam(model.parameters(), lr=5e-5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Tokenization

In [23]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

    Tokenizing the whole dataset

In [24]:
#Tokenize the dataset to the correct input for the transformer model.
def tokenize(batch):
    return tokenizer(batch["text_clean"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
valid_dataset = valid_dataset.map(tokenize, batched=True, batch_size=len(valid_dataset))
test_dataset = test_ds.map(tokenize, batched=True, batch_size=len(test_ds))

Map: 100%|████████████████████████████████████████| 53941/53941 [00:49<00:00, 1099.87 examples/s]


## Training a classifier

In [25]:
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    save_strategy=save_strategy,
    save_steps=save_steps,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    logging_steps=logging_steps,
)

In [26]:
 trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [27]:
trainer.train() 

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.464900,0.473990,0.772929,[0.74212565 0.79715809],[0.78082495 0.76724688],[0.70708124 0.82949611]
2,0.420900,0.447152,0.785814,[0.77699925 0.79395766],[0.74871661 0.82267066],[0.80750251 0.76718136]
3,0.355200,0.443313,0.791060,[0.77441954 0.80541455],[0.77271819 0.80694714],[0.77612839 0.80388778]
4,0.334800,0.485304,0.792988,[0.7803957 0.80421518],[0.76540257 0.81850953],[0.79598796 0.79041153]
5,0.243000,0.565147,0.789818,[0.77098358 0.80579015],[0.77642607 0.80102858],[0.76561685 0.81060867]


TrainOutput(global_step=134850, training_loss=0.3745408267501022, metrics={'train_runtime': 30191.9149, 'train_samples_per_second': 71.462, 'train_steps_per_second': 4.466, 'total_flos': 2.8580900999221248e+17, 'train_loss': 0.3745408267501022, 'epoch': 5.0})

* Training loss: Difference between the predictons made by the model on the training dataset vs on the actual data.
* Validation loss: how well the model functions on unseen data.
* Accuracy: How much the model gets correct. number of correct Prediction / total number of predictions.
* F1: consider both precision and recall. 
* Precision: Accuracy of positive predictions. Percison TP = TP + FP. How often the model is correct.
* Recall: True positive rate. how many items the model gets correct from the total amount.

### Training loss decreases, valdiation loss increases = Overfitting

In [28]:
# Evaluate valdiation set
eval_result = trainer.evaluate(eval_dataset=valid_dataset)

In [29]:
for key, value in sorted(eval_result.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.7898181278852037

eval_f1 = [0.77098358 0.80579015]

eval_loss = 0.5651473999023438

eval_precision = [0.77642607 0.80102858]

eval_recall = [0.76561685 0.81060867]

eval_runtime = 232.1935

eval_samples_per_second = 232.302

eval_steps_per_second = 7.261



In [30]:
# Evaluate test data set
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [31]:
for key, value in sorted(test_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.7883798965536419

eval_f1 = [0.76997481 0.80405788]

eval_loss = 0.578072190284729

eval_precision = [0.77439099 0.80017082]

eval_recall = [0.76560872 0.80798289]

eval_runtime = 232.2529

eval_samples_per_second = 232.251

eval_steps_per_second = 7.259



In [32]:
trainer.save_model(model_dir + "_local") 

In [33]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local")

In [34]:
classifier.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [35]:
classifier("this does not need to be done fast")

[{'label': 'LABEL_0', 'score': 0.5640496611595154}]

In [36]:
classifier("this is super important")

[{'label': 'LABEL_0', 'score': 0.9732745885848999}]

In [37]:
classifier("this bug has super high impact on the project")

[{'label': 'LABEL_0', 'score': 0.9805421829223633}]

## Important to delete large objects to free memory 
del train_dataset

In [38]:
del valid_dataset

In [39]:
del model

In [40]:
# Free cache
torch.cuda.empty_cache()

In [41]:
!nvidia-smi

Tue Nov  7 18:46:44 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-PCIE-40GB           On | 00000000:01:00.0 Off |                    0 |
| N/A   57C    P0              240W / 250W|  27028MiB / 40960MiB |    100%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--